# Getting Started with OPERA DSWx Products
## Visualizing OPERA Dynamic Surface Water Extent (DSWx) product and Harmonized Landsat-8 Sentinel-2 (HLS) data in the Cloud using CMR's SpatioTemporal Asset Catalog (CMR-STAC)
### This tutorial demonstrates how to work with the OPERA DSWx Products and query the input HLS Landsat 8 ([HLSL30.002](https://doi.org/10.5067/HLS/HLSL30.002)) and Sentinel-2 ([HLSS30.002](https://doi.org/10.5067/HLS/HLSS30.002)) data products.

---    

### Data Used in the Example  

- **Daily 30 meter (m) global HLS Sentinel-2 Multi-spectral Instrument Surface Reflectance - [HLSS30.002](https://doi.org/10.5067/HLS/HLSS30.002)**
    - _The HLSS30 product provides 30 m Nadir normalized Bidirectional Reflectance Distribution Function (BRDF)-Adjusted Reflectance (NBAR) and is derived from Sentinel-2A and Sentinel-2B MSI data products._  
     - **Science Dataset (SDS) layers:**  
        - B8A (NIR Narrow)  
        - B04 (Red)  
        - B02 (Blue)  

- **Daily 30 meter (m) global HLS Landsat-8 OLI Surface Reflectance - [HLSL30.002](https://doi.org/10.5067/HLS/HLSL30.002)**
    - _The HLSL30 product provides 30 m Nadir normalized Bidirectional Reflectance Distribution Function (BRDF)-Adjusted Reflectance (NBAR) and is derived from Landsat-8 OLI data products._  
     - **Science Dataset (SDS) layers:**  
        - B05 (NIR)  
        - B04 (Red)  
        - B02 (Blue)  
        
---

## Before Starting this Tutorial  

A [NASA Earthdata Login](https://urs.earthdata.nasa.gov/) account is required to download the data used in this tutorial. You can create an account at the link provided.

You will will also need to have a netrc file set up in your home directory in order to successfully run the code below. Check out the `Setting up a netrc File` section in the [README](https://git.earthdata.nasa.gov/projects/LPDUR/repos/hls-tutorial/browse/README.md).

---
## 1. Getting Started <a id="getstarted"></a>

### 1.1 Import Packages <a id="1.1"></a>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from pystac_client import Client  
from pystac_client import ItemSearch
from pystac.item import Item
from typing import Dict, Any
import json

import matplotlib.pyplot as plt
from datetime import datetime
from tqdm import tqdm

from shapely.geometry import box
from shapely.geometry import shape
from shapely.ops import transform

import numpy as np
import pandas as pd
import geopandas as gpd
from skimage import io

from osgeo import gdal
import rasterio as rio
from rasterio.mask import mask
from rasterio.enums import Resampling
from rasterio.shutil import copy

import pyproj
from pyproj import Proj

import xarray as xr
import geoviews as gv
from cartopy import crs
import hvplot.xarray
import holoviews as hv
gv.extension('bokeh', 'matplotlib')

import leafmap.leafmap as leafmap
import ipyleaflet

import sys
sys.path.append('../')
from src.dist_utils import *

import tempfile
from http import cookiejar
from urllib import request
from urllib.parse import urlencode

### 1.2 Set up Working Environment <a id="1.2"></a>

In [3]:
inDir = os.getcwd()
os.chdir(inDir)

#### 1.2.1 Authentication

In [4]:
#check for netrc credentials. If not present, user is prompted to enter them. Take care to enter them correctly.
authenticators = check_netrc()

netrc exists and includes NASA Earthdata login credentials.


In [5]:
#create password manager for authentication
make_manager(authenticators)

### Draw AOI

In [104]:
m = ipyleaflet.Map(
    basemap=ipyleaflet.basemaps.Esri.WorldTopoMap,
    center=(37, -100),
    zoom=3.5,
    crs=ipyleaflet.projections.EPSG3857
    )

dc = ipyleaflet.DrawControl(
    polygon={"shapeOptions": {"color": "blue"}},
    rectangle={"shapeOptions": {"color": "blue"}},
    circlemarker={},
    polyline={}
)

print('Select an Area of Interest using the tools on the left side of the map.')
dc.on_draw(handle_draw)
m.add_control(dc)
display(m)

Select an Area of Interest using the tools on the left side of the map.


Map(center=[37, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [105]:
ll = dc.last_draw['geometry']['coordinates'][0][0]
ur = dc.last_draw['geometry']['coordinates'][0][2]

### 1.3 Initialize the Search Parameters <a id="1.3"></a>

In [214]:
#define aoi
ll = dc.last_draw['geometry']['coordinates'][0][0]
ur = dc.last_draw['geometry']['coordinates'][0][2]
aoi = box(ll[0], ll[1], ur[0], ur[1])

#define period of interest
start_date = datetime(2023, 6, 1)                                       # in 2022-01-01 00:00:00 format
#stop_date = f"{datetime.today().strftime('%Y-%m-%d')} 23:59:59"         # in 2022-01-01 00:00:00 format
stop_date = datetime(2023, 6, 30)

#define thresholds
overlap_threshold = 10                                                  # in percent
cloud_cover_threshold = 10                                              # in percent

print(f"Search between {start_date} and {stop_date}")
print(f"With AOI: {aoi.__geo_interface__}")

Search between 2023-06-01 00:00:00 and 2023-06-30 00:00:00
With AOI: {'type': 'Polygon', 'coordinates': (((-97.478943, 30.083355), (-97.478943, 30.519681), (-98.028259, 30.519681), (-98.028259, 30.083355), (-97.478943, 30.083355)),)}


### 1.4 Initialize STAC API and search for HLS Collections in LP DAAC Catalogs <a id="1.4"></a>

In [107]:
## Uncomment to inspect parameters inside ItemSearch
#ItemSearch??

In [108]:
stac = 'https://cmr.earthdata.nasa.gov/stac/'    # CMR-STAC API Endpoint
api = Client.open(f'{stac}/LPCLOUD/')
hls_collections = ['HLSL30.v2.0', 'HLSS30.v2.0']

search_params = {"collections": hls_collections,
                 "intersects": aoi.__geo_interface__,
                 "datetime": [start_date, stop_date],
                 "max_items": 1000}
search_hls = api.search(**search_params)

### 1.5 Compute for spatial overlap with respect to defined AOI <a id="1.5"></a>

In [109]:
# Function to calculate percentage overlap
def intersection_percent(item: Item, aoi: Dict[str, Any]) -> float:
    '''The percentage that the Item's geometry intersects the AOI. An Item that
    completely covers the AOI has a value of 100.
    '''
    geom_item = shape(item.geometry)
    geom_aoi = shape(aoi)
    intersected_geom = geom_aoi.intersection(geom_item)
    intersection_percent = (intersected_geom.area * 100) / geom_aoi.area

    return intersection_percent

In [110]:
# Filter datasets based on spatial overlap and cloud cover
intersects_geometry = aoi.__geo_interface__

#Check percent overlap values
print("Percent overlap before filtering: ")
print([f"{intersection_percent(i, intersects_geometry):.2f}" for i in search_hls.items()])

#Check cloud cover values
print("")
print("Cloud cover before filtering: ")
print([f"{i.properties['eo:cloud_cover']}" for i in search_hls.items()])

# Apply spatial overlap and cloud cover threshold
hls_filtered = (
    i for i in search_hls.items() if intersection_percent(i, intersects_geometry) > overlap_threshold \
        and i.properties['eo:cloud_cover'] <= cloud_cover_threshold
)

Percent overlap before filtering: 
['87.57', '30.95', '73.38', '5.03', '87.57', '30.95', '87.57', '30.95', '72.46', '4.41', '87.57', '73.07', '4.83', '30.95', '87.57', '87.57', '30.95', '71.46', '3.76', '87.57', '30.95', '87.57', '30.95', '72.00', '4.10']

Cloud cover before filtering: 
['52', '19', '38', '65', '34', '73', '59', '82', '26', '7', '96', '50', '100', '0', '2', '4', '4', '3', '3', '11', '2', '14', '0', '29', '5']


In [111]:
# Inspect the items inside the filltered query
hls_data = list(hls_filtered)
# Inspect one data
hls_data[0].to_dict()

{'type': 'Feature',
 'stac_version': '1.0.0',
 'id': 'HLS.L30.T14RNU.2023167T170219.v2.0',
 'properties': {'eo:cloud_cover': 0,
  'datetime': '2023-06-16T17:02:19.317000Z',
  'start_datetime': '2023-06-16T17:02:19.317Z',
  'end_datetime': '2023-06-16T17:02:43.216Z'},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-97.8647821, 29.7373348],
    [-97.8533412, 30.7280047],
    [-98.750134, 30.7328295],
    [-98.9995863, 29.7422038],
    [-97.8647821, 29.7373348]]]},
 'links': [{'rel': 'self',
   'href': 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/HLSL30.v2.0/items/HLS.L30.T14RNU.2023167T170219.v2.0'},
  {'rel': 'parent',
   'href': 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/HLSL30.v2.0'},
  {'rel': 'collection',
   'href': 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/HLSL30.v2.0'},
  {'rel': <RelType.ROOT: 'root'>,
   'href': 'https://cmr.earthdata.nasa.gov/stac//LPCLOUD/',
   'type': <MediaType.JSON: 'application/json'>,
   'title': 'LPCLOUD'},

In [112]:
## Print search information
# Total granules
print(f"Total granules after search filter: {len(hls_data)}")

#Check percent overlap values
print("Percent-overlap: ")
print([f"{intersection_percent(i, intersects_geometry):.2f}" for i in hls_data])

#Check cloud cover values
print("")
print("Cloud cover: ")
print([f"{i.properties['eo:cloud_cover']}" for i in hls_data])

Total granules after search filter: 7
Percent-overlap: 
['30.95', '87.57', '87.57', '30.95', '71.46', '30.95', '30.95']

Cloud cover: 
['0', '2', '4', '4', '3', '2', '0']


### 1.6 Visualize the AOI and HLS granules' footprint based on search parameters <a id="1.6"></a>

In [113]:
geom_df = []
for d,_ in enumerate(hls_data):
    geom_df.append(shape(hls_data[d].geometry))

geom_granules = gpd.GeoDataFrame({'geometry':geom_df})
granules_poly = gv.Polygons(geom_granules).opts(line_color='blue', color=None)

# Use geoviews to combine a basemap with the shapely polygon of our Region of Interest (ROI)
base = gv.tile_sources.EsriImagery.opts(width=1000, height=1000)

# Get the user-specified aoi
geom_aoi = shape(intersects_geometry)
aoi_poly = gv.Polygons(geom_aoi).opts(line_color='yellow', color=None)

# Plot using geoviews wrapper
granules_poly*base*aoi_poly

:Overlay
   .Polygons.I  :Polygons   [Longitude,Latitude]
   .WMTS.I      :WMTS   [Longitude,Latitude]
   .Polygons.II :Polygons   [Longitude,Latitude]

In [114]:
# GDAL configurations used to successfully access LP DAAC Cloud Assets via vsicurl 
gdal.SetConfigOption('GDAL_HTTP_COOKIEFILE','~/cookies.txt')
gdal.SetConfigOption('GDAL_HTTP_COOKIEJAR', '~/cookies.txt')
gdal.SetConfigOption('GDAL_DISABLE_READDIR_ON_OPEN','EMPTY_DIR')
gdal.SetConfigOption('CPL_VSIL_CURL_ALLOWED_EXTENSIONS','TIF')

In [115]:
print([f"{i.properties['datetime']}" for i in hls_data])

['2023-06-16T17:02:19.317000Z', '2023-06-16T17:02:19.317Z', '2023-06-18T17:25:27.844Z', '2023-06-18T17:25:31.446Z', '2023-06-20T17:15:32.361Z', '2023-06-23T17:25:31.358Z', '2023-06-24T17:02:30.571Z']


### 1.7 Build a Geodatabase <a id="1.7"></a>

In [116]:
hls_data_df = []
for item in hls_data:
    item.to_dict()
    ID = item.id
    dat = item.datetime.strftime('%Y-%m-%d')
    spatial_coverage = intersection_percent(item, intersects_geometry)
    cloud_cover = item.properties['eo:cloud_cover']
    geom = item.geometry
    thumb = item.assets['browse'].href
    
    collection = item.assets['B01'].href.split('/')[4]
    if collection == 'HLSS30.020':
        band_links = [item.assets['B8A'].href, item.assets['B04'].href, 
                      item.assets['B03'].href, item.assets['B02'].href]
    elif collection == 'HLSL30.020':
        band_links = [item.assets['B05'].href, item.assets['B04'].href,
                      item.assets['B03'].href, item.assets['B02'].href]
    
    hls_data_df.append([dat,ID,collection,cloud_cover,geom,spatial_coverage,band_links,thumb])

hls_data_df = pd.DataFrame(hls_data_df, columns = ['Date', 'ID', 'Collection', 'Cloud_Cover','Footprint','Spatial_Coverage','Band_Links','Thumbnail'])


In [175]:
footprints = []
hls_data_df['Footprint']
for fp in hls_data_df['Footprint']:
    footprints.append(Polygon(fp['coordinates'][0]))

In [182]:
gp = gv.Polygons(footprints)
aoi_poly = gv.Polygons(geom_aoi)
gp*base*aoi_poly

:Overlay
   .Polygons.I  :Polygons   [Longitude,Latitude]
   .WMTS.I      :WMTS   [Longitude,Latitude]
   .Polygons.II :Polygons   [Longitude,Latitude]

In [117]:
# Find cloud cover equal to 0
hls_data_df[hls_data_df['Cloud_Cover'] == 0]

,Date,ID,Collection,Cloud_Cover,Footprint,Spatial_Coverage,Band_Links,Thumbnail
0,2023-06-16,HLS.L30.T14RNU.2023167T170219.v2.0,HLSL30.020,0,"{'type': 'Polygon', 'coordinates': [[[-97.8647...",30.946205,[https://data.lpdaac.earthdatacloud.nasa.gov/l...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...
6,2023-06-24,HLS.L30.T14RNU.2023175T170230.v2.0,HLSL30.020,0,"{'type': 'Polygon', 'coordinates': [[[-97.8647...",30.946205,[https://data.lpdaac.earthdatacloud.nasa.gov/l...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...


In [118]:
hls_data_df

,Date,ID,Collection,Cloud_Cover,Footprint,Spatial_Coverage,Band_Links,Thumbnail
0,2023-06-16,HLS.L30.T14RNU.2023167T170219.v2.0,HLSL30.020,0,"{'type': 'Polygon', 'coordinates': [[[-97.8647...",30.946205,[https://data.lpdaac.earthdatacloud.nasa.gov/l...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...
1,2023-06-16,HLS.L30.T14RPU.2023167T170219.v2.0,HLSL30.020,2,"{'type': 'Polygon', 'coordinates': [[[-97.0268...",87.569205,[https://data.lpdaac.earthdatacloud.nasa.gov/l...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...
2,2023-06-18,HLS.S30.T14RPU.2023169T170901.v2.0,HLSS30.020,4,"{'type': 'Polygon', 'coordinates': [[[-97.4225...",87.569205,[https://data.lpdaac.earthdatacloud.nasa.gov/l...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...
3,2023-06-18,HLS.S30.T14RNU.2023169T170901.v2.0,HLSS30.020,4,"{'type': 'Polygon', 'coordinates': [[[-97.8647...",30.946205,[https://data.lpdaac.earthdatacloud.nasa.gov/l...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...
4,2023-06-20,HLS.S30.T14RPU.2023171T165849.v2.0,HLSS30.020,3,"{'type': 'Polygon', 'coordinates': [[[-96.8309...",71.459715,[https://data.lpdaac.earthdatacloud.nasa.gov/l...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...
5,2023-06-23,HLS.S30.T14RNU.2023174T170849.v2.0,HLSS30.020,2,"{'type': 'Polygon', 'coordinates': [[[-97.8647...",30.946205,[https://data.lpdaac.earthdatacloud.nasa.gov/l...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...
6,2023-06-24,HLS.L30.T14RNU.2023175T170230.v2.0,HLSL30.020,0,"{'type': 'Polygon', 'coordinates': [[[-97.8647...",30.946205,[https://data.lpdaac.earthdatacloud.nasa.gov/l...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...


In [193]:
### Testing here some attempts to stream the HLS tiles and merge them 
def merge_rasters(input_files, output_file):
    from rasterio.merge import merge
    # Open the input rasters and retrieve metadata
    src_files = [rasterio.open(file) for file in input_files]
    meta = src_files[0].meta
    mosaic, out_trans = merge(src_files)

    with rasterio.open(output_file, 'w', **meta) as dst:
       dst.write(mosaic)
    # Create the output raster dataset
    # with rasterio.open(output_file, 'w', **meta) as dst:
    #     # Iterate over input rasters and write data to the output
    #     for src in src_files:
    #         for _, window in dst.block_windows(1):
    #             data = src.read(window=window)
    #             dst.write(data, window=window)

    #Close the input rasters
    for src in src_files:
        src.close()

    return mosaic

In [194]:
input_files = []
for i in hls_data_df['Band_Links']:
    input_files.append(i[0])

In [195]:
len(input_files)

7

In [196]:
import rasterio
mosaic = merge_rasters(input_files, 'test_merge.tif')

In [201]:
import matplotlib.pyplot as plt

In [205]:
mosaic[0].shape

(3660, 6994)

In [213]:
from rasterio.plot import show
show(mosaic)

<Axes: >

### 1.8 Quickview of two granules <a id="1.8"></a>
Browse image files are loaded into memory.

In [49]:
getdatabydates = ['2022-07-21']               # my preferred dates
#my_hls = hls_data_df[(hls_data_df['Date'].isin(getdatabydates)) & (hls_data_df['Collection'].isin(['HLSS30.020']))]
my_hls = hls_data_df[(hls_data_df['Date'].isin(getdatabydates)) & (hls_data_df['Collection'].isin(['HLSL30.020']))]
my_hls

,Date,ID,Collection,Cloud_Cover,Footprint,Spatial_Coverage,Band_Links,Thumbnail


In [50]:
# Load the two dates in memory
#before_img = io.imread(my_hls.iloc[0].Band_Links[0])
#after_img = io.imread(my_hls.iloc[1].Thumbnail)

In [51]:
# import tempfile
# from http import cookiejar
# from urllib import request
# from urllib.parse import urlencode

# # Create a password manager to deal with the 401 response that is returned from
# # Earthdata Login
# password_manager = request.HTTPPasswordMgrWithDefaultRealm()
# password_manager.add_password(None, "https://urs.earthdata.nasa.gov", netrc(netrcDir).authenticators(urs)[0], netrc(netrcDir).authenticators(urs)[2])
# # Create a cookie jar for storing cookies. This is used to store and return
# # the session cookie given to use by the data server (otherwise it will just
# # keep sending us back to Earthdata Login to authenticate). Ideally, we
# # should use a file based cookie jar to preserve cookies between runs. This
# # will make it much more efficient.
# cookie_jar = cookiejar.CookieJar()
# # Install all the handlers.
# opener = request.build_opener(
#    request.HTTPBasicAuthHandler(password_manager),
#    request.HTTPCookieProcessor(cookie_jar))
# request.install_opener(opener)
# # Create and submit the request. There are a wide range of exceptions that
# # can be thrown here, including HTTPError and URLError. These should be
# # caught and handled.
# #url = 'https://n5eil01u.ecs.nsidc.org/MOSA/MYD10_L2.061/2002.07.04/MYD10_L2.A2002185.2040.061.2020071133128.hdf'
# # url = my_hls.iloc[0].Band_Links[0]
# # myrequest = request.Request(url)
# # response = request.urlopen(myrequest)

# # print(response.code)
# # # Check if status is OK
# # if response.code != 200:
# #   raise ValueError
# # response.begin()
# # local_output = tempfile.NamedTemporaryFile(delete=False)
# # while True:
# #   chunk = response.read()
# #   if chunk: 
# #     local_output.file.write(chunk)
# #   else:
# #     break
# # local_output.file.close()
# # print(local_output.name)

In [52]:
# ### Boiling the above down to what seems "necessary" for leaflet map to successfully download the earthdata data
# import tempfile
# from http import cookiejar
# from urllib import request
# from urllib.parse import urlencode

# # Create a password manager to deal with the 401 response that is returned from
# # Earthdata Login
# password_manager = request.HTTPPasswordMgrWithDefaultRealm()
# password_manager.add_password(None, "https://urs.earthdata.nasa.gov", netrc(netrcDir).authenticators(urs)[0], netrc(netrcDir).authenticators(urs)[2])

# # Create a cookie jar for storing cookies. This is used to store and return
# # the session cookie given to use by the data server (otherwise it will just
# # keep sending us back to Earthdata Login to authenticate). Ideally, we
# # should use a file based cookie jar to preserve cookies between runs. This
# # will make it much more efficient.
# cookie_jar = cookiejar.CookieJar()
# # Install all the handlers.
# opener = request.build_opener(
#    request.HTTPBasicAuthHandler(password_manager),
#    request.HTTPCookieProcessor(cookie_jar))
# request.install_opener(opener)

In [53]:
# Load the COG for each band to arrays
print("loading cog bands into memory")
for band in hls_data_df.Band_Links[0]:
    with rio.open(band, mode="r") as src:
        
        #get information neccessary to write file
        transform = src.transform
        crs = src.crs
        
        #load bands into arrays
        if band.rsplit('.', 2)[-2] == 'B05' or band.rsplit('.', 2)[-2] == 'B8A':      # NIR index
            nir = src.read(1)
        elif band.rsplit('.', 2)[-2] == 'B04':    # red index
            red = src.read(1)
        elif band.rsplit('.', 2)[-2] == 'B03':    # green index
            green = src.read(1)
        elif band.rsplit('.', 2)[-2] == 'B02':    # blue index
            blue = src.read(1)
print("COGS have been loaded as arrays")

loading cog bands into memory
COGS have been loaded as arrays


In [ ]:
# # Load the COG for each band to memory
# print("loading cog bands into memory")
# for band in hls_data_df.Band_Links[0]:
#     print(band.rsplit('.', 2)[-2])
#     if band.rsplit('.', 2)[-2] == 'B05' or band.rsplit('.', 2)[-2] == 'B8A':      # NIR index
#         nir = rio.open(band)
#     elif band.rsplit('.', 2)[-2] == 'B04':    # red index
#         red = rio.open(band)
#     elif band.rsplit('.', 2)[-2] == 'B03':    # green index
#         green = rio.open(band)
#     elif band.rsplit('.', 2)[-2] == 'B02':    # blue index
#         blue = rio.open(band)
# print("The COGs have been loaded into memory!")

In [ ]:
# #load to arrays
# nir_a = nir.read(1)
# red_a = red.read(1)
# green_a = green.read(1)
# blue_a = blue.read(1)

In [54]:
#compute NDVI
ndvi = (nir - red) / (nir + red)
mask = (ndvi > 0) & (ndvi < 1)
ndvi_cor = np.ma.masked_array(ndvi, ~mask)

In [55]:
cube = np.stack((nir, red, green, blue))

In [56]:
#write NDVI and RGB to COG files that will be stored locally in working directory
NRGB_dataset = rio.open(
    'cube_test_auto.tif',
    'w',
    driver='GTiff',
    height=cube.shape[1],
    width=cube.shape[2],
    count=4,
    dtype=cube.dtype,
    crs=crs,
    transform=transform
)

NDVI_dataset = rio.open(
    'ndvi_thresh_test_auto.tif',
    'w',
    driver='GTiff',
    height=ndvi_cor.shape[0],
    width=ndvi_cor.shape[1],
    count=1,
    dtype=ndvi_cor.dtype,
    crs=crs,
    transform=transform
)

In [57]:
NRGB_dataset.write(cube)
NDVI_dataset.write(ndvi_cor,1)

In [58]:
NRGB_dataset.close()
NDVI_dataset.close()

In [ ]:
### Local files
leafmap.split_map(
    left_layer="cube.tif",
    right_layer="ndvi_thresh.tif",
    left_label="2001",
    right_label="2016",
    label_position="bottom",
    center=[42.0, -122.3],
    zoom=5,
)

In [ ]:
leafmap.download_from_url("https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T10TEM.2022202T185058.v2.0/HLS.L30.T10TEM.2022202T185058.v2.0.B05.tif", "t1.tif")
leafmap.download_from_url("https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T10TEM.2022266T185119.v2.0/HLS.L30.T10TEM.2022266T185119.v2.0.B05.tif", "t2.tif")

In [ ]:
### Local files
leafmap.split_map(
    left_layer="false_color_7_21_2022.tif",
    right_layer="false_color_9_23_2022.tif",
    left_label="07/21/2022",
    right_label="09/23/2022",
    label_position="bottom",
    center=[42.0, -122.3],
    zoom=10,
)

## would be cool to have multiple leaflet maps which allows turning on HLS-RGB, HLS-NDVI, and then has a slider for going forward and backward in time